# Architecture

More:
* [The Twelve-Factor App.ipynb](./The%20Twelve-Factor%20App.ipynb)

# Patterns
* [Patterns.ipynb](./Patterns.ipynb)

## Reactor
* [wikipedia](https://en.wikipedia.org/wiki/Reactor_pattern)
* Schmidt, Douglas C. **Reactor: An Object Behavioral Pattern forDemultiplexing and Dispatching Handles for Synchronous Events**. 1995.

> Reactor设计模式处理处理由一个或多个客户端并发生成的服务请求. 
> 每个应用中的服务由几个方法组成, 用单独的事件处理器表示服务, 事件处理器负责分发(dispatch)服务相关的请求. 
> 一个初始的分发器负责分发事件处理器, 并管理事件处理的注册.
> 服务请求的分离由一个同步的事件分离器(demultiplexer)执行.

> Reactor/Dispatcher/Notifier

terms:
* reactor: 反应器
* multiplexer: 多路复用器
* demultiplexer: 信号分离器
* dispatcher: 分发器
* notifier: 通知器

处理问题: C10K problem, 并发事件处理问题
* 网络socket
* 硬件IO
* 文件系统/数据库访问
* IPC(Inter Process Communication)
* message passing systems

组件结构:
* 句柄: 一个特定IO/数据请求的标识符和接口. 通常是socket, 文件描述符等.
* 信号分离器(Demultiplexer): 一个可以有效的监控句柄状态的事件通知器, 然后通知其他子系统一个相关状态变更. 通常是`select()`系统调用, 也可以是epoll, kqueue, IOCP.
* 分发器: 响应式应用的实际事件循环(event loop), 维护事件处理器的注册, 在事件发生时调用相应的处理器.
* 事件处理器: 请求处理器, 是处理一类服务请求的特定逻辑. 建议动态注册为回调. 不使用多线程的reactor默认在与分发器相同的线程中调用请求处理器.
* 事件处理器接口: 一个表示通用的事件处理器的属性和方法的抽象接口.

<img src="https://upload.wikimedia.org/wikipedia/commons/2/2a/Reactor_Pattern_-_UML_2_Component_Diagram.svg" width="800"/>

初始化时序:
* 初始化分发器/事件循环.
* 注册事件处理器.
* 打开句柄: 关联分发器和信号分离器.
* 开始事件循环: 等待事件发生.

运行时序:
* (客户端)通过句柄传递请求.
* 信号分离器检测到句柄状态变化, 通知分发器有事件发生.
* 分发器调用事件处理器: 携带句柄参数.
* 事件处理器读写句柄, 处理请求.
* 分发器开始下一次事件循环.


application: Netty
* Handle: Channel
  * 打开句柄: 注册到Selector(NIO), 注册到EventLoop.
* Dispatcher: NioEventLoop, EpollEventLoop
  * 绑定到固定的一个线程.
* Demultiplexer: Selector, `io.netty.channel.epoll.Native#newEpollCreate`
* Event Handler: ChannelHandler

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/8/87/ReactorPattern_-_UML_2_Sequence_Diagram.svg/771px-ReactorPattern_-_UML_2_Sequence_Diagram.svg.png" width="800"/>

* Doug Lea. **Scalable IO in Java**. [link](http://gee.cs.oswego.edu/dl/cpjslides/nio.pdf), [link](https://cdn.jsdelivr.net/gh/fuxyzz/cdn/files/nio.pdf)

网络服务的基本结构
- 读取请求, 解码请求, 处理请求的服务, 编码响应, 发送响应

方案: 
- 每个处理器有自己的线程
- 事件驱动
  - Reactor单线程版本: reactor/dispatch和acceptor共用一个线程.
- 多线程
  - 工作者线程: 将非IO处理卸载到其他线程中执行
    - IO处理: acceptor, 读取请求, 发送响应. 非IO处理: 解码请求, 处理请求的服务, 编码响应.
    - 线程池: 工作者线程, 任务队列.
  - 多个Reactor线程:  
    - mainReactor: 处理acceptor, 成功建立连接后注册到subReactor. - 处理接入认证
    - subReactor: 处理读取请求, 发送响应, 将非IO处理卸载到工作者线程中处理.

`java.nio`:
- Bufffer, ByteBuffer
- Channel, SelectableChannel, SocketChannel, ServerSocketChannel, FileChannel
- Selector, SelectionKey

## Proactor
* [wikipedia](https://en.wikipedia.org/wiki/Proactor_pattern)
* Pyarali, Irfan / Tim Harrison / Schmidt, Douglas C. / Jordan, Thomas D. **Proactor: An Object Behavioral Pattern for Demultiplexing**. 1997.

> 本文中的前摄器模式描述了如何组织应用和系统以高效的利用操作系统支持的异步机制.
> 当应用调用一个异步操作室, 操作系统代为执行操作. 这允许应用有多个同时运行的操作, 而不需要有对应数量的线程.
> 因此, 前摄器模式简化了并发编程, 通过较少的线程和利用操作系统异步操作支持, 提供性能.

terms:
* proactor: 前摄器
* completion handler: 完成处理器

前摄器模式是同步的reactor模式的异步版本变种.

组件结构:
* Asynchronous Operation: 异步操作. 由操作系统代为执行, 从应用视角该操作是异步的. 
  * 例: Async_Read, Async_Write, Async_Accpet
* Proactive Initiator: 前摄初始化器, 是应用发起异步操作的实体, 向异步操作处理器中注册完成处理器和完成分发器.
  * 例: Web服务器应用的主线程
* Asynchronous Operation Processor: 异步操作处理器, 通常由操作系统实现.
* Completion Dispatcher: 异步操作完成事件分发器. 异步操作完成时, 异步操作处理器将应用通知委托给分发器.
  * 例: 通知队列
* Completion Handler: 异步操作完成事件处理器.
  * 例: Acceptor, HTTP Handler

运行时序:
* 应用使用前摄初始化器发起异步操作, 创建/指定相应的句柄, 并向异步操作处理器注册完成处理器和完成分发器.
* 异步操作处理器执行异步操作.
* 异步操作完成时, 异步操作处理器调用对应的完成分发器, 传递异步操作的结果和对应的完成处理器.
* 完成分发器调用完成处理器, 传递异步操作的结果.

<img src="https://upload.wikimedia.org/wikipedia/commons/6/61/Proactor.VSD_SequenceDiagram.png" width="800"/>

# High Performance Methodology

ref: [一文了解高性能架构和系统设计经验](https://www.51cto.com/article/745546.html)

1. 前端层面
* 数据与加载, 本地缓存, CDN加速, 长连接, 异步请求

2. 编码实现层面
* 多线程, 多协程
* 无锁设计
* 数据序列化
* 池化: 线程池, 连接池, 内存池, 对象池
* 异步: IO, 消息队列
* 并发流程: 针对无依赖关系的任务

3. 单机架构
* IO多路复用
  * Reactor架构模式: 非阻塞同步网络
  * Proactor架构模式: 非阻塞异步网络

1. 系统架构
* 合理的架构: 高内聚低耦合, 单一职责 - 分层, 模块化的设计
* 微服务: RPC(序列化协议, 压缩), 服务治理(注册, 发现, 负载均衡)
* 网络模型: 同步/异步, 连接池
* 负载均衡: 策略包括随机, 轮询, hash, 权限, 连接数
* 消息队列: 异步逻辑, 解耦, 削峰填谷
* 延时队列: 定时, 回调, 重试
* 任务队列: 异步, 线程池
* 缓存: 读多写少场景, 提高读性能; 本地缓存 + 分布式缓存; 缓存预热
* NoSQL: 数据分区/分片, 分库分表, 读写分离

1. 基础设施层
* 机器配置
* 资源使用: 容量规划, 监控, 扩容

1. 运维部署
* 容器化弹性部署

1. 性能测试
* 压测, 分析, 优化